<a href="https://colab.research.google.com/github/vilsonrodrigues/ComputerVision/blob/master/ITA_Challenge_2021-TrafficLightDetection/TrafficLightDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseado em: 

https://github.com/AlexeyAB/darknet

https://medium.com/analytics-vidhya/train-a-custom-yolov4-object-detector-using-google-colab-61a659d4868

## Step 0 - Criar Pastas no Drive

Monte o seu Drive e em seguida crie uma pasta para o seu projeto. Veja que antes eu criei uma pasta mais geral para os projetos de detecção

In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/detection

mkdir: cannot create directory ‘/content/drive/MyDrive/detection’: File exists


In [ ]:
!mkdir /content/drive/MyDrive/detection/TrafficLightDetection

mkdir: cannot create directory ‘/content/drive/MyDrive/detection/TrafficLightDetection’: File exists


## Step 1 - Clonando o Repositório Darknet 

Baixando o repositório que contêm a implementação da YOLOv4 e inserindo na pasta

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection
!git clone https://github.com/AlexeyAB/darknet

/content/drive/MyDrive/detection/TrafficLightDetection
Cloning into 'darknet'...
remote: Enumerating objects: 15054, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 15054 (delta 19), reused 40 (delta 16), pack-reused 15002
Receiving objects: 100% (15054/15054), 13.48 MiB | 6.74 MiB/s, done.
Resolving deltas: 100% (10218/10218), done.
Checking out files: 100% (2028/2028), done.


Chegando se tudo ocorreu bem

In [ ]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
darknet  data


## Step 2 - Montagem da Estrutura do Projeto

Usando o software LabelImg você pode etiquetar suas imagens para a detecção. A criação de um bom dataset é a parte mais fundamental em um modelo, se você criar um dataset mal anotado/enviesado o modelo vai aprender esse viés e quando for a produção vai performar mal

As imagens precisam ser quadradas e o tamanho ser múltiplo de 32. As do dataset tem uma dimensão 1280x720x3

Copiando a pasta das imagens para a pasta do projeto

In [ ]:
!cp -r -n /content/drive/MyDrive/Datasets/TrafficLightDetection/data /content/drive/MyDrive/detection/TrafficLightDetection/

Baixando os arquivos necessários

In [ ]:
#%%capture
%cd /content/drive/MyDrive/detection/TrafficLightDetection
!wget -P data/ https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/obj.names
!wget -P data/ https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/obj.data
!wget -P data/dev/ https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/dev.txt
!wget -P data/train/ raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/train.txt
#!wegt -P data/ https://github.com/vilsonrodrigues/ComputerVision/blob/master/ITA_Challenge_2021-TrafficLightDetection/utils.py

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
--2021-05-29 00:02:55--  https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/obj.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25 [text/plain]
Saving to: ‘data/obj.names’

obj.names           100%[===================>]      25  --.-KB/s    in 0s      

2021-05-29 00:02:55 (907 KB/s) - ‘data/obj.names’ saved [25/25]



In [ ]:
#%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
#!wget -P data/ https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/utils.py

In [ ]:
#%cd /content/drive/MyDrive/detection/TrafficLightDetection/data
#!ls

/content/drive/MyDrive/detection/TrafficLightDetection/data
dev  obj.data  obj.names  train


In [ ]:
#%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
#!python utils.py

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data


In [ ]:
#ai voce coloca dentro das pastas dev e train

In [ ]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection

darknet  data


In [ ]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection/data/

dev  obj.data  obj.names  train


Primeiro vamos mover a pasta para dentro do diretório para facilitar a execução


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/
!cp -r -n data/ darknet/

/content/drive/MyDrive/detection/TrafficLightDetection


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/
!ls

/content/drive/MyDrive/detection/TrafficLightDetection
darknet  data


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
9k.tree     dog.jpg	 horses.jpg		   obj.data	     scream.jpg
coco9k.map  eagle.jpg	 imagenet.labels.list	   obj.names	     train
coco.names  giraffe.jpg  imagenet.shortnames.list  openimages.names  voc.names
dev	    goal.txt	 labels			   person.jpg


## Step 3 - Habilitando CUDA e GPU

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/drive/MyDrive/detection/TrafficLightDetection/darknet


## Step 4 - Construindo Darkent

In [ ]:
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

## Step 5 - Calculando Novas Âncoras

Dado que foi mudada a resolução padrão do yolov4 é necessário recalcular as 9 âncoras

In [ ]:
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 512 -height 512

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 512, height = 512 
 read labels from 756 images 
 loaded 	 image: 756 	 box: 1181
 all loaded. 

 calculating k-means++ ...

 iterations = 34 


counters_per_class = 678, 28, 419, 56

 avg IoU = 83.10 % 

Saving anchors to the file: anchors.txt 
anchors =   3,  8,   3, 11,   4, 10,   4, 14,   5, 17,   7, 23,   9, 30,  13, 42,  18, 61
^C


Com as novas âncoras calculadas, foi alterado no .cfg do github e agora vamos introduzi-lo no darknet

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4.cfg
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4_2.cfg
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4_3.cfg
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4_4.cfg
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4_5.cfg
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet
--2021-05-27 01:45:20--  https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13380 (13K) [text/plain]
Saving to: ‘yolov4.cfg’

yolov4.cfg          100%[===================>]  13.07K  --.-KB/s    in 0.001s  

2021-05-27 01:45:20 (11.4 MB/s) - ‘yolov4.cfg’ saved [13380/13380]

3rdparty		darknet.py	       obj
anchors.txt		darknet_video.py       README.md
backup			data		       results
build			image_yolov3.sh        scripts
build.ps1		image_yolov4.sh        src
cfg			include		       uselib
cmake			json_mjpeg_streams.sh  vcpkg.json
CMakeLists.txt		libdarknet.so	       video_yolov3.sh


## Step 6 - Treinamento

Iniciando treino do zero

In [ ]:
!./darknet detector train data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -dont_show -map

A saída de streaming foi truncada nas últimas 5000 linhas.
 total_bbox = 1258168, rewritten_bbox = 0.192423 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.814498), count: 21, class_loss = 0.815732, iou_loss = 191.408737, total_loss = 192.224472 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.746534), count: 1, class_loss = 0.180506, iou_loss = 5.506100, total_loss = 5.686605 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 1258190, rewritten_bbox = 0.192419 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.755240), count: 7, class_loss = 0.010609, iou_loss = 69.217270, total_loss = 69.227882 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_lo

Para restatar o treino caso a sessão caia, basta reconectar o drive, habilitar o cuda e o opencv e fazer o make para construir o yolov4. Depois executa a célula abaixo

In [ ]:
!./darknet detector train data/obj.data yolov4_2.cfg backup/yolov4_last.weights -dont_show -map

## Step 7 - Salvando os resultados

Com um threshold limitante de 0.25

In [ ]:
# Gerando o arquivo train.txt
import os
os.getcwd()
collection = 'data/teste/traffic_light_test'

%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet

file = open('data/teste/teste.txt', "w+")

for filename in os.listdir(collection):
  if(filename[-3:] != 'txt'):
    file.write('data/teste/traffic_light_test/' + filename + '\n')

file.close()

/content/drive/MyDrive/detection/TrafficLightDetection/darknet


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!./darknet detector test data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -thresh 0.25 -dont_show -save_labels < data/teste/teste.txt

A saída de streaming foi truncada nas últimas 5000 linhas.
data/teste/traffic_light_test/25380.png: Predicted in 24.366000 milli-seconds.
red: 30%
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
data/teste/traffic_light_test/25422.png: Predicted in 24.314000 milli-seconds.
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
data/teste/traffic_light_test/25432.png: Predicted in 24.307000 milli-seconds.
red: 38%
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
data/teste/traffic_light_test/25412.png: Predicted in 24.297000 milli-seconds.
Enter Image Path:  Detection layer: 139 - type = 28 
 Detection layer: 150 - type = 28 
 Detection layer: 161 - type = 28 
data/teste/traffic_light_test/25382.png: Predicted in 24.340000 milli-seconds.
red: 27%
Enter Image Path:  Detectio

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/traffic_light_test/
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/traffic_light_test
24068.png  26450.png  28832.png  31214.png  33596.png  35978.png  38360.png
24068.txt  26450.txt  28832.txt  31214.txt  33596.txt  35978.txt  38360.txt
24070.png  26452.png  28834.png  31216.png  33598.png  35980.png  38362.png
24070.txt  26452.txt  28834.txt  31216.txt  33598.txt  35980.txt  38362.txt
24072.png  26454.png  28836.png  31218.png  33600.png  35982.png  38364.png
24072.txt  26454.txt  28836.txt  31218.txt  33600.txt  35982.txt  38364.txt
24074.png  26456.png  28838.png  31220.png  33602.png  35984.png  38366.png
24074.txt  26456.txt  28838.txt  31220.txt  33602.txt  35984.txt  38366.txt
24076.png  26458.png  28840.png  31222.png  33604.png  35986.png  38368.png
24076.txt  26458.txt  28840.txt  31222.txt  33604.txt  35986.txt  38368.txt
24078.png  26460.png  28842.png  31224.png  33606.png  35988.png  38370.png
24078.txt  26460.txt  28842.txt  31224.txt  33606.txt  35988.txt  38370

Aplicando o utils2 para salvar as predições e o nome do arquivo em um só txt

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/traffic_light_test/
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/utils2.py
!python utils2.py

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/traffic_light_test
--2021-05-29 07:57:58--  https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/utils2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 633 [text/plain]
Saving to: ‘utils2.py’

utils2.py           100%[===================>]     633  --.-KB/s    in 0s      

2021-05-29 07:57:58 (18.6 MB/s) - ‘utils2.py’ saved [633/633]

Traceback (most recent call last):
  File "utils2.py", line 16, in <module>
    line = data.readline()
  File "/usr/lib/python3.7/codecs.py", line 319, in decode
    def decode(self, input, final=False):
KeyboardInterrupt


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/lote
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/lote
24068.png  24078.txt  24090.png  24100.txt  24112.png  24122.txt  24134.png
24068.txt  24080.png  24090.txt  24102.png  24112.txt  24124.png  24134.txt
24070.png  24080.txt  24092.png  24102.txt  24114.png  24124.txt  24136.png
24070.txt  24082.png  24092.txt  24104.png  24114.txt  24126.png  24136.txt
24072.png  24082.txt  24094.png  24104.txt  24116.png  24126.txt  24138.png
24072.txt  24084.png  24094.txt  24106.png  24116.txt  24128.png  24138.txt
24074.png  24084.txt  24096.png  24106.txt  24118.png  24128.txt  results.txt
24074.txt  24086.png  24096.txt  24108.png  24118.txt  24130.png  utils2.py
24076.png  24086.txt  24098.png  24108.txt  24120.png  24130.txt
24076.txt  24088.png  24098.txt  24110.png  24120.txt  24132.png
24078.png  24088.txt  24100.png  24110.txt  24122.png  24132.txt


In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/lote
!python utils2.py

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste/lote


Deixando no padrão do desafio

In [ ]:
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste
df = pd.read_table('results.txt', delim_whitespace=True, names=('filename','status','x','y','w','h'))
df

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste


,filename,status,x,y,w,h
0,24068.png,2,0.2610,0.2787,0.0113,0.0207
1,24652.png,0,0.1150,0.5762,0.0163,0.0554
2,24690.png,0,0.0634,0.5553,0.0128,0.0477
3,24714.png,0,0.0218,0.5478,0.0075,0.0264
4,24766.png,0,0.6242,0.4774,0.0081,0.0341
...,...,...,...,...,...,...
6456,40440.png,0,0.1760,0.0608,0.0382,0.1077
6457,40442.png,0,0.1663,0.0541,0.0391,0.0982
6458,40562.png,0,0.3252,0.5386,0.0092,0.0234
6459,40618.png,0,0.4425,0.0083,0.0207,0.0168


In [ ]:
df.head(2)

,filename,status,x,y,w,h
0,24068.png,2,0.261,0.2787,0.0113,0.0207
1,24652.png,0,0.115,0.5762,0.0163,0.0554


In [ ]:
df2 = df
df2

,filename,status,x,y,w,h
0,24068.png,2,0.2610,0.2787,0.0113,0.0207
1,24652.png,0,0.1150,0.5762,0.0163,0.0554
2,24690.png,0,0.0634,0.5553,0.0128,0.0477
3,24714.png,0,0.0218,0.5478,0.0075,0.0264
4,24766.png,0,0.6242,0.4774,0.0081,0.0341
...,...,...,...,...,...,...
6456,40440.png,0,0.1760,0.0608,0.0382,0.1077
6457,40442.png,0,0.1663,0.0541,0.0391,0.0982
6458,40562.png,0,0.3252,0.5386,0.0092,0.0234
6459,40618.png,0,0.4425,0.0083,0.0207,0.0168


In [ ]:
dh = 720
dw = 1280

l = int(( df['x'] - df['w'] / 2) * dw)
r = int((df['x'] + df['w'] / 2) * dw)
t = int((df['y'] - df['h'] / 2) * dh)
b = int((df['y'] + df['h'] / 2) * dh)

TypeError: ignored

Substituindo as classes para os nomes

In [ ]:
df['status'] = df['status'].replace({0:'green',1:'yellow',2:'red',3:'off'})

In [ ]:
#'filename','status','x_min','x_max','y_min','y_max'

In [ ]:
df.to_csv('results.csv', index=0)
df.head()

,filename,status,x,y,w,h
0,24068.png,red,0.261,0.2787,0.0113,0.0207


## Step 8 - Visualizando Métricas

Função auxiliar

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Só funcionará se o treinamento não for interrompido

In [ ]:
imShow('chart.png')

AttributeError: ignored

Checando a **mAP (mean average precision)**. Atenção ao selecionar os melhores pesos. Hipoteticamente pode ser na 10000 interação como está abaixo

In [ ]:
%cd backup
!ls

/content/drive/MyDrive/detection/VowelDetectionLibras-yolov4/darknet/backup-weights
yolov4_best.weights  yolov4_last.weights


In [ ]:
!./darknet detector map data-detection/obj.data yolov4.cfg backup/yolov4_best.weights -points 0

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 610, cudnn_half = 0, GPU: Tesla P4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    224 x 224 x   3 ->  224 x 224 x  32 0.087 BF
   1 conv     64       3 x 3/ 2    224 x 224 x  32 ->  112 x 112 x  64 0.462 BF
   2 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   3 route  1 		                           ->  112 x 112 x  64 
   4 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   5 conv     32       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  32 0.051 BF
   6 conv     64       3 x 3/ 1    112 x 112 x  32 ->  112 x 112 x  64 0.462 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 112 x 112 x  64 0.001 BF
   8 conv  

## Step 9 - Testando o Detector

Primeiro vamos alterar o batch para 1 e as subdivisions para 1 também

In [ ]:
! sed -i 's / batch = 64 / batch = 1 /' yolov4.cfg 
! sed -i 's / subdivisions = 16 / subdivisions = 1 /' yolov4.cfg 

sed: -e expression #1, char 11: unknown option to `s'
sed: -e expression #1, char 18: unknown option to `s'


Executando o detector para 1 imagem

In [ ]:
!./darknet detector test data-detection/obj.data yolov4.cfg backup/yolov4_best.weights mask_test_images/image1.jpg -thresh 0.25
imShow('predictions.jpg')

Rodando em um vídeo

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet

!./darknet detector demo data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -dont_show Sao Paulo.mp4 -i 0 -out_filename mask_test_videos/finalresult.avi 

## Step 10 - Testando o nosso modelo com um conjunto de imagens

In [ ]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data/teste
!unzip traffic_light_test.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: traffic_light_test/30736.png  
  inflating: traffic_light_test/30738.png  
  inflating: traffic_light_test/30740.png  
  inflating: traffic_light_test/30742.png  
  inflating: traffic_light_test/30744.png  
  inflating: traffic_light_test/30746.png  
  inflating: traffic_light_test/30748.png  
  inflating: traffic_light_test/30750.png  
  inflating: traffic_light_test/30752.png  
  inflating: traffic_light_test/30754.png  
  inflating: traffic_light_test/30756.png  
  inflating: traffic_light_test/30758.png  
  inflating: traffic_light_test/30760.png  
  inflating: traffic_light_test/30762.png  
  inflating: traffic_light_test/30764.png  
  inflating: traffic_light_test/30766.png  
  inflating: traffic_light_test/30768.png  
  inflating: traffic_light_test/30770.png  
  inflating: traffic_light_test/30772.png  
  inflating: traffic_light_test/30774.png  
  inflating: traffic_light_test/30776.png  
  inflating: traf

In [ ]:
# Gerando o arquivo test.txt

%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data

import os
os.getcwd()
collection = 'teste/lote'

file = open('teste/teste.txt', "w+")

for filename in os.listdir(collection):
  #if(filename[-3:] != 'txt'):
  file.write('data/teste/lote/' + filename + '\n')

file.close()

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data


In [ ]:
!ls

9k.tree     giraffe.jpg		      obj.data		teste
coco9k.map  goal.txt		      obj.names		teste.txt
coco.names  horses.jpg		      openimages.names	train
dev	    imagenet.labels.list      person.jpg	utils.py
dog.jpg     imagenet.shortnames.list  result.txt	voc.names
eagle.jpg   labels		      scream.jpg


In [ ]:
# Testando 
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!./darknet detector test data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -thresh 0.25 -dont_show < data/teste/teste.txt > data/result.txt

/content/drive/MyDrive/detection/TrafficLightDetection/darknet
 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 700, cudnn_half = 1, GPU: Tesla V100-SXM2-16GB 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    608 x 384 x   3 ->  608 x 384 x  32 0.403 BF
   1 conv     64       3 x 3/ 2    608 x 384 x  32 ->  304 x 192 x  64 2.152 BF
   2 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   3 route  1 		                           ->  304 x 192 x  64 
   4 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   5 conv     32       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  32 0.239 BF
   6 conv     64       3 x 3/ 1    304 x 192 x  32 ->  304 x 192 x  64 2.152 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 192 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF


In [ ]:
!./darknet detector test data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -ext_output data/teste/25614.png

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 700, cudnn_half = 1, GPU: Tesla V100-SXM2-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 384 x   3 ->  608 x 384 x  32 0.403 BF
   1 conv     64       3 x 3/ 2    608 x 384 x  32 ->  304 x 192 x  64 2.152 BF
   2 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   3 route  1 		                           ->  304 x 192 x  64 
   4 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   5 conv     32       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  32 0.239 BF
   6 conv     64       3 x 3/ 1    304 x 192 x  32 ->  304 x 192 x  64 2.152 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 192 x  64 0.004 BF

In [ ]:
!./darknet detector test data/obj.data yolov4_5.cfg backup/yolov4_5_best.weights -ext_output data/teste/25614.png

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 700, cudnn_half = 1, GPU: Tesla V100-SXM2-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 384 x   3 ->  608 x 384 x  32 0.403 BF
   1 conv     64       3 x 3/ 2    608 x 384 x  32 ->  304 x 192 x  64 2.152 BF
   2 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   3 route  1 		                           ->  304 x 192 x  64 
   4 conv     64       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  64 0.478 BF
   5 conv     32       1 x 1/ 1    304 x 192 x  64 ->  304 x 192 x  32 0.239 BF
   6 conv     64       3 x 3/ 1    304 x 192 x  32 ->  304 x 192 x  64 2.152 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 304 x 192 x  64 0.004 BF